In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
#from sklearn.cross_validation import train_test_split
import re,os
import json
import ast,nltk
import datetime
from stemming.porter2 import stem
from collections import defaultdict
from string import punctuation
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import logging
import time
from gensim.models import Word2Vec,KeyedVectors
from sklearn.preprocessing import Imputer

In [2]:
training_data = []
# Articles that has category given
with open('articles.json','r') as data_file:
    line = data_file.readline()
    while line:
        line = ast.literal_eval(line)
        training_data.append(line)
        line = data_file.readline()
    data_file.close()

In [3]:
testing_data = []
# Articles that has no category given
with open('articles_wo.json','r') as data_file:
    line = data_file.readline()
    while line:
        line = ast.literal_eval(line)
        testing_data.append(line)
        line = data_file.readline()
    data_file.close()

In [4]:
# function to parse sentences to words and remove stopwords from sentence
def sentence_to_wordlist( sentence, remove_stopwords=True ):
    text = re.sub("[^a-zA-Z]"," ",sentence)
    words = text.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return(words)

In [5]:
# function to parse text to sentences using tokenizer mentioned above
def text_to_sentences(text,tokenizer,remove_stopwords=True):
    raw_sentences = tokenizer.tokenize(text.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append( sentence_to_wordlist( raw_sentence, \
              remove_stopwords ))
    return sentences

In [6]:
categories_dict = {'automobiles' : 0, 'business' : 1, 'computing' : 2, 'criminals' : 3, 'entertainment' : 4, 'fashion' : 5,\
                  'foods' : 6, 'health' : 7, 'lifestyle' : 8, 'pbusiness' : 9, 'pfashion' : 10, 'phealth' : 11,\
                  'plifestyle' : 12, 'politics' : 13, 'ptechnology' : 14, 'ptravel' : 15, 'science' : 16, 'sports' : 17,\
                  'technology' : 18, 'travel' : 19}

In [7]:
#words = []
classes = []
documents = []
#no_classes = []
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
# loop through each sentence in our training data
for pattern in training_data:
    # tokenize each word in the sentence
    #words.extend(nltk.word_tokenize(pattern['desc']))
    documents += [[stem(str(word)) for word in sentence] \
                        for sentence in [text_to_sentences(pattern['desc'], tokenizer)]]
    classes.append(categories_dict[pattern['category']])

In [8]:
for pattern in testing_data:
    documents += [[stem(str(word)) for word in sentence] \
                          for sentence in [text_to_sentences(pattern['desc'],tokenizer)]]

In [9]:
def build_model(documents):
    # Import the built-in logging module and configure it so that Word2Vec 
    # creates nice output messages
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
        level=logging.INFO)

    # Set values for various parameters
    num_features = 300    # Word vector dimensionality                      
    min_word_count = 2  # Minimum word count                        
    num_workers = 4       # Number of threads to run in parallel
    context = 4           # Context window size                                                                                    
    downsampling = 1e-5   # Downsample setting for frequent words
    # Initialize and train the model (this will take some time)

    print "Training model..."
    model = Word2Vec(documents, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context)

    # If you don't plan to train the model any further, calling 
    # init_sims will make the model much more memory-efficient.
    model.init_sims(replace=False)

    # It can be helpful to create a meaningful model name and 
    # save the model for later use. You can load it later using Word2Vec.load()
    model_name = "300features_2minwords_4context"
    model.save(model_name)
    #model.save_word2vec_format(model_name,binary=False)
    return model

In [10]:
def makeFeatureVec(words,model,num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float128")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #sorted_set = set(model.sort_vocab())
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [11]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float128")
    # 
    # Loop through the reviews
    for review in reviews:
    # Print a status message every 1000th review
        if counter%1000. == 0.:
            print "Review %d of %d" % (counter, len(reviews))

    # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
    # Increment the counter
        counter = counter + 1.
    return reviewFeatureVecs

In [12]:
model = build_model(documents)
DataVecs = getAvgFeatureVecs( documents, model, 300 )

2017-10-31 15:24:53,794 : INFO : collecting all words and their counts
2017-10-31 15:24:53,796 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-10-31 15:24:53,957 : INFO : PROGRESS: at sentence #10000, processed 204500 words, keeping 182852 word types
2017-10-31 15:24:53,969 : INFO : collected 190351 word types from a corpus of 213395 raw words and 10852 sentences
2017-10-31 15:24:53,970 : INFO : Loading a fresh vocabulary


Training model...


2017-10-31 15:24:54,182 : INFO : min_count=2 retains 14692 unique words (7% of original 190351, drops 175659)
2017-10-31 15:24:54,184 : INFO : min_count=2 leaves 37736 word corpus (17% of original 213395, drops 175659)
2017-10-31 15:24:54,248 : INFO : deleting the raw counts dictionary of 190351 items
2017-10-31 15:24:54,255 : INFO : sample=0.001 downsamples 2 most-common words
2017-10-31 15:24:54,257 : INFO : downsampling leaves estimated 36502 word corpus (96.7% of prior 37736)
2017-10-31 15:24:54,258 : INFO : estimated required memory for 14692 words and 300 dimensions: 42606800 bytes
2017-10-31 15:24:54,324 : INFO : resetting layer weights
2017-10-31 15:24:54,569 : INFO : training model with 4 workers on 14692 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=4
2017-10-31 15:24:54,570 : INFO : expecting 10852 sentences, matching count from corpus used for vocabulary survey
2017-10-31 15:24:55,163 : INFO : worker thread finished; awaiting finish of 3 more t

Review 0 of 10852
Review 1000 of 10852
Review 2000 of 10852
Review 3000 of 10852
Review 4000 of 10852
Review 5000 of 10852
Review 6000 of 10852
Review 7000 of 10852
Review 8000 of 10852
Review 9000 of 10852
Review 10000 of 10852


In [13]:
X_train = Imputer().fit_transform(DataVecs[:10369])
Y_train = np.array(classes)

In [19]:
#test_model = build_model(test_documents)
#TestDataVecs = getAvgFeatureVecs(test_documents,test_model,300)
X_test = Imputer().fit_transform(DataVecs[10369:])
#print X_test[0]

In [37]:
bdt_real = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2),n_estimators=600,learning_rate=1.5,algorithm="SAMME")
bdt_real.fit(X_train, Y_train)
Y_test = bdt_real.predict(X_test)

In [15]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

/home/dell/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [18]:
for category in predictions:
    print categories_dict.keys()[int(category)]
#print predictions

fashion
computing
fashion
fashion
fashion
fashion
fashion
fashion
fashion
computing
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
computing
fashion
fashion
fashion
fashion
fashion
computing
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
fashion
computing
computing
fashion
computing
fashion
fashion
fashion
fashion
fashion
fashion
computing
fashion


In [27]:
print X_train[0]

[ -1.24384760e-03  -8.47065938e-04   1.89919946e-03   1.42334960e-04
  -2.17245036e-03  -1.59490202e-03   9.85398423e-04   1.65537160e-03
   7.76215136e-04  -1.92861352e-03   1.44635812e-03   5.24335541e-04
  -4.12896392e-03   3.75122228e-03  -2.30566271e-03   3.55496269e-03
   1.27599348e-03  -3.65242362e-03  -5.84987225e-04  -1.73114933e-03
   1.80758057e-03  -2.76248436e-03  -9.16886667e-04   3.50919156e-03
   8.95717472e-04  -1.19994834e-03  -2.60477688e-03  -7.71814230e-04
  -3.16000881e-03  -8.57082894e-04  -4.99296002e-05   3.73968773e-03
  -2.54886091e-03   2.06646742e-04   2.77502247e-03   2.07541842e-03
   1.26061076e-03   9.52588860e-04   7.49359257e-04   2.44473334e-03
   2.38667859e-03  -4.46007616e-04   2.72727321e-03  -4.23072453e-03
   2.39738310e-03   2.46637396e-03   3.21137975e-03   2.58726068e-04
  -1.65540478e-03   1.01002050e-03   3.95442569e-03  -2.16846392e-03
  -3.52181832e-03  -4.52782493e-04   3.73213075e-03  -2.38172198e-03
   1.36116287e-05   8.03649076e-04

In [22]:
def get_error_rate(pred, Y):
    return sum(pred != Y) / float(len(Y))

def generic_clf(Y_train, X_train, X_test, clf):
    clf.fit(X_train,Y_train)
    pred_train = clf.predict(X_train)
    pred_test = clf.predict(X_test)
    #return get_error_rate(pred_train, Y_train), \
    #    get_error_rate(pred_test, Y_test)
    return pred_test
    
def adaboost_clf(Y_train, X_train, X_test, M, clf):
    n_train, n_test = len(X_train), len(X_test)
    # Initialize weights
    w = np.ones(n_train) / n_train
    pred_train, pred_test = [np.zeros(n_train), np.zeros(n_test)]
    
    for i in range(M):
        # Fit a classifier with the specific weights
        clf.fit(X_train, Y_train, sample_weight = w)
        pred_train_i = clf.predict(X_train)
        pred_test_i = clf.predict(X_test)
        # Indicator function
        miss = [int(x) for x in (pred_train_i != Y_train)]
        # Equivalent with 1/-1 to update weights
        miss2 = [x if x==1 else -1 for x in miss]
        # Error
        err_m = np.dot(w,miss) / sum(w)
        # Alpha
        alpha_m = 0.5 * np.log( (1 - err_m) / float(err_m))
        # New weights
        w = np.multiply(w, np.exp([float(x) * alpha_m for x in miss2]))
        # Add to prediction
        pred_train = [sum(x) for x in zip(pred_train, 
                                          [x * alpha_m for x in pred_train_i])]
        pred_test = [sum(x) for x in zip(pred_test, 
                                         [x * alpha_m for x in pred_test_i])]
    
    pred_train, pred_test = np.sign(pred_train), np.sign(pred_test)
    # Return error rate in train and test set
    #return get_error_rate(pred_train, Y_train), \
    #       get_error_rate(pred_test, Y_test)
    return pred_test

In [25]:
clf_tree = DecisionTreeClassifier(max_depth = 1, random_state = 1)
pred_test = adaboost_clf(Y_train, X_train, X_test, 50, clf_tree)

In [26]:
print pred_test

[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1